<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
%%writefile get_data.py

import os
import requests
import zipfile

# URL для загрузки данных
data_url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
data_path = "data/pizza_steak_sushi"

# Функция для загрузки и распаковки данных
def download_and_extract_data(url, path):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)
        zip_path = os.path.join(path, "pizza_steak_sushi.zip")

        with open(zip_path, "wb") as file:
            response = requests.get(url)
            file.write(response.content)

        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(path)

        os.remove(zip_path)
        print(f"[INFO] Data downloaded and extracted to {path}")
    else:
        print(f"[INFO] Data already exists at {path}")

# Запуск функции
if __name__ == "__main__":
    download_and_extract_data(data_url, data_path)

Writing get_data.py


In [2]:
# Example running of get_data.py
!python get_data.py

[INFO] Data downloaded and extracted to data/pizza_steak_sushi


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [3]:
%%writefile train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import argparse
import torch
from torchvision import transforms

import data_setup
import engine
import model_builder
import utils

def parse_args():
    parser = argparse.ArgumentParser(description="Train a PyTorch model with custom hyperparameters.")

    parser.add_argument("--num_epochs", type=int, default=10, help="Number of epochs to train the model")
    parser.add_argument("--batch_size", type=int, default=32, help="Batch size for the DataLoader")
    parser.add_argument("--hidden_units", type=int, default=10, help="Number of hidden units in the model")
    parser.add_argument("--learning_rate", type=float, default=0.001, help="Learning rate for the optimizer")
    parser.add_argument("--train_dir", type=str, default="data/pizza_steak_sushi/train", help="Directory for training data")
    parser.add_argument("--test_dir", type=str, default="data/pizza_steak_sushi/test", help="Directory for testing data")

    return parser.parse_args()

def main():
    args = parse_args()

    NUM_EPOCHS = args.num_epochs
    BATCH_SIZE = args.batch_size
    HIDDEN_UNITS = args.hidden_units
    LEARNING_RATE = args.learning_rate
    TRAIN_DIR = args.train_dir
    TEST_DIR = args.test_dir

    print(f"[INFO] Training with: epochs={NUM_EPOCHS}, batch_size={BATCH_SIZE}, hidden_units={HIDDEN_UNITS}, learning_rate={LEARNING_RATE}")
    print(f"[INFO] Training data path: {TRAIN_DIR}")
    print(f"[INFO] Testing data path: {TEST_DIR}")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[INFO] Using device: {device}")

    data_transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor()
    ])

    train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
        train_dir=TRAIN_DIR,
        test_dir=TEST_DIR,
        transform=data_transform,
        batch_size=BATCH_SIZE
    )

    model = model_builder.TinyVGG(
        input_shape=3,
        hidden_units=HIDDEN_UNITS,
        output_shape=len(class_names)
    ).to(device)

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    engine.train(
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        epochs=NUM_EPOCHS,
        device=device
    )

    utils.save_model(
        model=model,
        target_dir="models",
        model_name="tinyvgg_model.pth"
    )
    print(f"[INFO] Model saved to models/tinyvgg_model.pth")

if __name__ == "__main__":
    main()


Writing train.py


In [4]:
# Example running of train.py
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

[INFO] Training with: epochs=5, batch_size=128, hidden_units=128, learning_rate=0.0003
[INFO] Training data path: data/pizza_steak_sushi/train
[INFO] Testing data path: data/pizza_steak_sushi/test
[INFO] Using device: cpu
  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1041 | train_acc: 0.3396 | test_loss: 1.0955 | test_acc: 0.3333
 20% 1/5 [00:31<02:07, 31.80s/it]Epoch: 2 | train_loss: 1.0914 | train_acc: 0.3651 | test_loss: 1.0782 | test_acc: 0.4800
 40% 2/5 [01:03<01:34, 31.56s/it]Epoch: 3 | train_loss: 1.0732 | train_acc: 0.4443 | test_loss: 1.0510 | test_acc: 0.4933
 60% 3/5 [01:33<01:02, 31.04s/it]Epoch: 4 | train_loss: 1.0288 | train_acc: 0.5786 | test_loss: 1.0278 | test_acc: 0.4400
 80% 4/5 [02:05<00:31, 31.29s/it]Epoch: 5 | train_loss: 0.9658 | train_acc: 0.5260 | test_loss: 1.0206 | test_acc: 0.4400
100% 5/5 [02:36<00:00, 31.31s/it]
[INFO] Saving model to: models/tinyvgg_model.pth
[INFO] Model saved to models/tinyvgg_model.pth


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [11]:
%%writefile predict.py
"""
Predicts the class of an input image using a pretrained PyTorch model.
"""

import argparse
import torch
import torchvision.transforms as transforms
from PIL import Image
import model_builder

def parse_args():
    parser = argparse.ArgumentParser(description="Predict the class of an input image using a pretrained model.")

    parser.add_argument("--image", type=str, required=True, help="Path to the image file for prediction.")
    parser.add_argument("--model_path", type=str, default="/content/models/tinyvgg_model.pth", help="Path to the saved model file.")

    return parser.parse_args()

def load_model(model_path, device):
    model = model_builder.TinyVGG(input_shape=3, hidden_units=128, output_shape=3).to(device)
    model.load_state_dict(torch.load(model_path))
    return model

def predict_image(image_path, model, class_names, device):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor()
    ])
    image = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.inference_mode():
        output = model(image)
        probabilities = torch.softmax(output, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item()

    return class_names[predicted_label], probabilities[0][predicted_label].item()

def main():
    args = parse_args()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    class_names = ["pizza", "steak", "sushi"]

    model = load_model(args.model_path, device)

    pred_class, pred_prob = predict_image(args.image, model, class_names, device)

    print(f"[INFO] Predicted class: {pred_class}, Probability: {pred_prob:.3f}")

if __name__ == "__main__":
    main()

Overwriting predict.py


In [13]:
# Example running of predict.py
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

[INFO] Predicted class: pizza, Probability: 0.358
